# Tarea5: Series de Taylor

Fecha de envío del PR inicial: **viernes 21 de abril**

Fecha de aceptación del PR: **martes 25 de abril, antes de la clase**

---

## Ejercicio 1

Implementen una nueva estructura paramétrica (`type`) que defina el tipo `Taylor`, donde el parámetro debe ser un subtipo de `Number`. Definan métodos que implementen las operaciones aritméticas básicas (`+`, `-`, `*`, `/`) y la igualdad (`==`). Esto deberá ser incluido en un módulo.

Incluyan pruebas (en el archivo "runtests_taylor.jl") para cada uno de los métodos que implementen.


In [1]:
"""
Definición de polinomios de Taylor, donde
...
"""
type Taylor{T<:Number}
    # código: 
    coeficientes::Array{Float64,1} #Array{T,1}
    grado::T #Para que el arreglo tenga las dimensiones del grado de la función
end

Taylor

In [2]:
Taylor(coeficientes,grado)= Taylor(promote(coeficientes,grado)...)#Taylor(coeficientes)= Taylor(promote(coeficientes)...)

Taylor{T<:Number}

In [3]:
v=[0.0,1.0,3.0]

3-element Array{Float64,1}:
 0.0
 1.0
 3.0

In [4]:
Taylor(v,9.0)

Taylor{Float64}([0.0,1.0,3.0],9.0)

In [5]:
function xtaylor(x)
    
    v = zeros(x) #Genera un array con 9 ceros.
    v=deleteat!(v,x) # retiramos el último de los ceros.
    v = push!(v,1.0) # agregamos en un 1 en el lugar del cero que se retiró. 
    #v=convert(v,Array{Number,1})

    return Taylor(v,x)
end


xtaylor (generic function with 1 method)

In [6]:
xtaylor(5)

Taylor{Int64}([0.0,0.0,0.0,0.0,1.0],5)

In [7]:
length(xtaylor(5).coeficientes)

5

In [ ]:
import Base: +, -, *, /, ==

# Aqui se implementan los métodos necesarios para cada función


In [ ]:
# Muestren que su código funciona con tests adecuados

using Base.Test
include("runtest_taylor.jl")

---

## Ejercicio 2

Obtengan las relaciones de recurrencia para las funciones $L(x) = \log\big(g(x)\big)$, $P_\alpha(x) = \big(g(x)\big)^\alpha$, $S(x) = \sin\big(g(x)\big)$, $C(x) = \cos\big(g(x)\big)$ usando el mismo procedimiento que arriba. Implementen métodos adecuados para estas funciones en el módulo, actuando sobre estructuras `Taylor` e incluyan pruebas necesarias en `runtest_taylor.jl`.